# 从原始数据开始预处理数据，轨迹保留长度为20-100，基站保留次数 > 30

In [63]:
import pandas as pd
import copy

# 读取轨迹数据
df_traj = pd.read_csv(f'/root/autodl-tmp/preprocess_data/original_data/t_dwt_person.csv')
# 读取基站数据
df_bs = pd.read_csv(f'/root/autodl-tmp/preprocess_data/original_data/t_dwt_uli_new_20250207.csv')


In [65]:
df_traj.head()

,c_number,c_uli,c_timestamp,c_day
0,a83411464ae86,1-4-178293763,1.728301e+09,20241007.0
1,a83411464ae86,1-4-178293770,1.728302e+09,20241007.0
2,a83411464ae86,1-4-178555139,1.728301e+09,20241007.0
3,a83411464ae86,1-4-178555148,1.728302e+09,20241007.0
4,a83411464ae86,1-4-178964740,1.728302e+09,20241007.0


In [66]:
# 统计c_uli列中每个值出现的次数
count_result = df_traj['c_uli'].value_counts()

# 筛选出出现次数≥30的值
filtered_result = count_result[count_result >= 30]

# 在df_traj中筛选出c_uli列的值在filtered_values中的行
df_traj_fliter_station = copy.copy(df_traj)

# 使用filtered_result的索引进行筛选
df_traj_fliter_station= df_traj_fliter_station[df_traj_fliter_station['c_uli'].isin(filtered_result.index)]

In [67]:
traj_fliter_station_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20391718 entries, 0 to 20542273
Data columns (total 4 columns):
 #   Column       Dtype  
---  ------       -----  
 0   c_number     object 
 1   c_uli        object 
 2   c_timestamp  float64
 3   c_day        float64
dtypes: float64(2), object(2)
memory usage: 777.9+ MB


In [79]:
# 假设traj_fliter_station_df是你当前的DataFrame
grouped = df_traj_fliter_station.groupby(['c_number', 'c_day'])

filtered_groups = []
for group_key, group_data in grouped:
    group_length = len(group_data)
    if 25 <= group_length <= 80:
        filtered_groups.append(group_data)

control_traj_df = pd.concat(filtered_groups, ignore_index=True)

In [80]:
control_traj_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18826944 entries, 0 to 18826943
Data columns (total 4 columns):
 #   Column       Dtype  
---  ------       -----  
 0   c_number     object 
 1   c_uli        object 
 2   c_timestamp  float64
 3   c_day        float64
dtypes: float64(2), object(2)
memory usage: 574.6+ MB


In [84]:
# 筛选轨迹中所有出现过的c_uli
traj_uli_list = control_traj_df['c_uli'].unique().tolist()
# 最后对基站文件进行过筛
final_df_bs = df_bs[df_bs['c_uli'].isin(traj_uli_list)]

In [85]:
final_df_bs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19351 entries, 0 to 43251
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   c_uli       19351 non-null  object 
 1   c_lac       19351 non-null  int64  
 2   c_ci        19351 non-null  int64  
 3   c_areacode  19351 non-null  int64  
 4   c_lat       19351 non-null  float64
 5   c_lon       19351 non-null  float64
dtypes: float64(2), int64(3), object(1)
memory usage: 1.0+ MB


In [88]:
new_data = []
for (c_number, c_day), group in grouped:
    sorted_group = group.sort_values('c_timestamp')
    traj = list(zip(sorted_group['c_uli'], sorted_group['c_timestamp']))
    new_data.append([c_number, c_day, traj])

final_traj_df = pd.DataFrame(new_data, columns=['c_number', 'c_day', 'traj'])

In [90]:
final_traj_df.head()

,c_number,c_day,traj
0,a83411464ae86,20241007.0,"[(1-4-188067383, 1728299383.0), (1-4-178295297..."
1,a834114664ee7,20241002.0,"[(1-4-188082491, 1727838016.0), (1-4-28465970,..."
2,a8341147567ec,20241004.0,"[(1-4-47212938, 1727976536.0), (1-4-50118919, ..."
3,a834114b8bbb6,20241007.0,"[(1-4-179201540, 1728289258.0), (1-4-47262898,..."
4,a834114c44dee,20241004.0,"[(3-4-179006730, 1728007295.0), (3-4-178665730..."


In [91]:
final_df_bs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19351 entries, 0 to 43251
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   c_uli       19351 non-null  object 
 1   c_lac       19351 non-null  int64  
 2   c_ci        19351 non-null  int64  
 3   c_areacode  19351 non-null  int64  
 4   c_lat       19351 non-null  float64
 5   c_lon       19351 non-null  float64
dtypes: float64(2), int64(3), object(1)
memory usage: 1.0+ MB
